---
title: "Sandbox"
jupyter: python3
code-fold: true 
execute:
  echo: false
  output: asis 
---

In [3]:
from datetime import datetime
import sys
import time
import requests
import html


api_url = "https://climatekg.semanticclimate.net/api.php"

def fetch_mediawiki_data(api_url, params, session):
    try:
        response = session.get(api_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from MediaWiki API: {e}")
        return None

def extract_items(data):
    items = []
    results = data.get("query", {}).get("results", {})
    
    for item_id, item_data in results.items():
        #print(item_data)
        
        

        multilang_fields = ["description"]
        entity_fields = ["subCategoryOf", "synonyms", "similarTo"]
        txt_fields = ["term","description_plain","vocabulary","acronym","tags","status"]
        item = {
            "id": item_id
        }

        for fieldname in txt_fields:
            if item_data["printouts"].get(fieldname, [{}]):
                #print(fieldname)
                item[fieldname] = item_data["printouts"].get(fieldname, [None])[0]
            else:
                item[fieldname] = ""
        
        for fieldname in entity_fields:
            if item_data["printouts"].get(fieldname, [{}]):
                #print(fieldname)
                item[fieldname] = str(item_data["printouts"].get(fieldname, [None]))
            else:
                item[fieldname] = ""

        for fieldname in multilang_fields:
            if item_data["printouts"].get(fieldname, [{}]):
                #print(fieldname)
                item[fieldname] = item_data["printouts"].get(fieldname, [{}])[0].get("Text", {}).get("item", [None])[0]
            else:
                item[fieldname] = ""
        
        if item["status"]:
            translate = {
                "Draft":"Entwurf",
                "Pending Review":"Review ausstehend",
                "Published":"Veröffentlicht",
                "Unpublished":"Unveröffentlicht"
            }
            for t in translate:
                if item["status"] == t:
                    item["status"] = translate[t]
        items.append(item)
    
    return items


def generate_markdown_table(items):
    markdown = '\n| ID | Begriff | Definition | Unterbegriff von |\n'
    markdown += '|---|---|---|---|\n'

    for item in items: 
        markdown += f'|{item["id"]}'
        markdown += f'|{item["term"]}'
        markdown += f'|{item["definition"]}'
        markdown += f'|{item["subCategoryOf"]}'
        markdown += '|\n'
    
    # Rows
    for item in items:
        html += '<tr>'
        html += f'<td>{item["id"]}</td>'
        html += f'<td>{item["term"]}</td>'
        html += f'<td>{item["definition"]}</td>'
        html += f'<td>{item["subCategoryOf"]}</td>'
        html += '</tr>'
    
    html += '</table>'
    return html

def generate_html_span(items, vocabulary):
    
    html = f''      
    for item in items:  
        html += f'''     
        
---

### <span class="gloss-term-all gloss-term">{item["term"]}</span> 

<span class="gloss-term-all gloss-definition">{item["description"]}</span>
        '''

        if item.get("description_plain"):
            html += f'''
**<span class="gloss-term-all gloss-cl-definition-label">Klartextbeschreibung:</span>**

<span class="gloss-term-all gloss-cl-definition">{item["description_plain"]}</span>
            ''' 

        if item.get("acronym"):
            html += f'''
**<span class="gloss-term-all gloss-acronym-label">Akronyme:</span>**  
      
<span class="gloss-term-all gloss-acronym">{item["acronym"]}</span>
            '''
        if item.get("status"):
            html += f'''
**<span class="gloss-term-all gloss-status-label">Status:</span>**  
      
<span class="gloss-term-all gloss-status">{item["status"]}</span>
            '''

        if item.get("tags"):
            html += f'''
**<span class="gloss-term-all gloss-tag-label">Tags:</span>**
      
<span class="gloss-term-all gloss-tag">{item["tags"]}</span>
            '''

        if item.get("similarTo"):
            html += f'''
**<span class="gloss-term-all gloss-similar-tag gloss-similar-label">Ähnlich:</span>**
      
<span class="gloss-term-all gloss-similar-tag">{item["similarTo"]}</span>
            '''

        if item.get("subCategoryOf"):
            html += f'''
**<span class="gloss-term-all gloss-subclass-tag gloss-subclass-label">Unterbegriff von:</span>**
      
<span class="gloss-term-all gloss-subclass-tag">{item["subCategoryOf"]}</span>
            '''

        if item.get("synonyms"):
            html += f'''
**<span class="gloss-term-all gloss-synonyme-tag gloss-synonyme-label">Synonyme:</span>**
      
<span class="gloss-term-all gloss-synonyme-tag">{item["synonyms"]}</span>
            
            '''
    return html

def login(api_url, username, password):
    session = requests.Session()
    
    # Step 1: GET request to fetch login token
    login_token_params = {
        'action': 'query',
        'meta': 'tokens',
        'type': 'login',
        'format': 'json'
    }
    response = session.get(api_url, params=login_token_params)
    response.raise_for_status()
    login_token = response.json()['query']['tokens']['logintoken']
    
    # Step 2: POST request to log in
    login_params = {
        'action': 'login',
        'lgname': username,
        'lgpassword': password,
        'lgtoken': login_token,
        'format': 'json'
    }
    response = session.post(api_url, data=login_params)
    response.raise_for_status()
    login_result = response.json()
    
    if login_result['login']['result'] == 'Success':
        #print('Logged in successfully')
        return session
    else:
        print('Failed to log in')
        sys.exit(1)

def get_glossary(vocabulary, session):

    params = {
        'action': 'ask',
        'query': f'[[Item:+]] [[has subobject::+]] [[P1::Term]] [[P18::{vocabulary}]]|?Wikibase description=description |?P26=description_plain |?P3=term |?P10.P3=subCategoryOf |?P20=status |?P21=acronym |?P22=tags |?P23=similarTo |?P18=vocabulary |?P24=synonyms |sort=P3',
        'format': 'json'
    }

    response = fetch_mediawiki_data(api_url, params, session)
    items = extract_items(response)
    
    if items:
        # Generate the HTML for the table
        html = '<html><head><link rel="stylesheet" href="glossary.css"></head><body>\n\n'
        #html += generate_html_table(items)
        html += generate_html_span(items,vocabulary)
        html += '\n</body></html>'
        print(html)

In [4]:
username = 'reader'  # Replace with your username
password = 'readonly'  # Replace with your password

session = login(api_url, username, password)
get_glossary("Sandbox", session)

<html><head><link rel="stylesheet" href="glossary.css"></head><body>

     
        
---

### <span class="gloss-term-all gloss-term">Begriff</span> 

<span class="gloss-term-all gloss-definition">Beschreibung</span>
        
**<span class="gloss-term-all gloss-status-label">Status:</span>**  
      
<span class="gloss-term-all gloss-status">Entwurf</span>
                 
        
---

### <span class="gloss-term-all gloss-term">Begriff2</span> 

<span class="gloss-term-all gloss-definition">Beschreibung</span>
        
**<span class="gloss-term-all gloss-acronym-label">Akronyme:</span>**  
      
<span class="gloss-term-all gloss-acronym">Akronym</span>
            
**<span class="gloss-term-all gloss-status-label">Status:</span>**  
      
<span class="gloss-term-all gloss-status">Entwurf</span>
            
**<span class="gloss-term-all gloss-tag-label">Tags:</span>**
      
<span class="gloss-term-all gloss-tag">Tags</span>
            
**<span class="gloss-term-all gloss-similar